In [ ]:
import socket
import threading
from PyQt5.QtWidgets import (
    QApplication, QVBoxLayout, QHBoxLayout, QPushButton, QWidget, QLabel, QListWidget, QLineEdit, QMessageBox
)
import sqlite3
from cryptography.fernet import Fernet
from bcrypt import hashpw, gensalt, checkpw

# Server configuration
HOST = '127.0.0.1'
PORT = 12346

clients = {}  # {client_socket: username}
usernames = {}  # {username: client_socket}
cipher_key = Fernet.generate_key()
cipher = Fernet(cipher_key)


def init_db():
    conn = sqlite3.connect("chat.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS users (
            username TEXT PRIMARY KEY,
            password TEXT
        )
    """)
    conn.commit()
    conn.close()


def broadcast(message, recipient=None, sender=None):
    """Send a message to a specific recipient or broadcast to all clients."""
    if recipient:
        target_client = usernames.get(recipient)
        if target_client:
            try:
                target_client.send(message)
            except:
                remove_client(target_client)
    else:
        for client in clients:
            if client != sender:
                try:
                    client.send(message)
                except:
                    remove_client(client)


def authenticate_user(client):
    """Authenticate or register a user."""
    client.send("AUTH".encode())
    creds = client.recv(1024).decode().split(":")
    username, password = creds[0], creds[1]

    conn = sqlite3.connect("chat.db")
    cursor = conn.cursor()
    cursor.execute("SELECT password FROM users WHERE username = ?", (username,))
    result = cursor.fetchone()

    if result:
        if checkpw(password.encode(), result[0].encode()):
            return username
        else:
            client.send("INVALID".encode())
            return None
    else:
        hashed_pw = hashpw(password.encode(), gensalt()).decode()
        cursor.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, hashed_pw))
        conn.commit()
        return username


def handle_client(client):
    """Handle communication with a connected client."""
    try:
        client.send(cipher_key)
        username = authenticate_user(client)

        if username:
            usernames[username] = client
            clients[client] = username
            broadcast(cipher.encrypt(f"{username} joined the chat.".encode()))

            while True:
                encrypted_message = client.recv(1024)
                message = cipher.decrypt(encrypted_message).decode()
                if ":" in message:
                    recipient, msg = message.split(":", 1)
                    broadcast(cipher.encrypt(f"{username} (to {recipient}): {msg}".encode()), recipient=recipient)
                else:
                    broadcast(cipher.encrypt(f"{username}: {message}".encode()), sender=client)
        else:
            client.close()
    except:
        remove_client(client)


def remove_client(client):
    """Remove a disconnected client."""
    if client in clients:
        username = clients.pop(client)
        usernames.pop(username, None)
        broadcast(cipher.encrypt(f"{username} has left the chat.".encode()))
        client.close()


class ServerGUI(QWidget):
    def __init__(self):
        super().__init__()
        self.init_ui()
        self.server_socket = None
        self.accept_thread = None
        self.is_running = False

    def init_ui(self):
        self.setWindowTitle("Chat Server")

        layout = QVBoxLayout()

        self.status_label = QLabel("Server is stopped.")
        layout.addWidget(self.status_label)

        self.client_list = QListWidget()
        layout.addWidget(self.client_list)

        button_layout = QHBoxLayout()

        self.start_button = QPushButton("Start Server")
        self.start_button.clicked.connect(self.start_server)
        button_layout.addWidget(self.start_button)

        self.stop_button = QPushButton("Stop Server")
        self.stop_button.clicked.connect(self.stop_server)
        self.stop_button.setEnabled(False)
        button_layout.addWidget(self.stop_button)

        layout.addLayout(button_layout)

        client_control_layout = QHBoxLayout()

        self.message_input = QLineEdit()
        self.message_input.setPlaceholderText("Type a message to broadcast...")
        client_control_layout.addWidget(self.message_input)

        self.broadcast_button = QPushButton("Broadcast Message")
        self.broadcast_button.clicked.connect(self.broadcast_message)
        client_control_layout.addWidget(self.broadcast_button)

        layout.addLayout(client_control_layout)

        self.setLayout(layout)

    def start_server(self):
        """Start the server."""
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server_socket.bind((HOST, PORT))
        self.server_socket.listen()
        self.is_running = True

        self.status_label.setText("Server is running...")
        self.start_button.setEnabled(False)
        self.stop_button.setEnabled(True)

        self.accept_thread = threading.Thread(target=self.accept_clients)
        self.accept_thread.start()

    def stop_server(self):
        """Stop the server."""
        self.is_running = False
        self.server_socket.close()
        self.status_label.setText("Server is stopped.")
        self.start_button.setEnabled(True)
        self.stop_button.setEnabled(False)

    def accept_clients(self):
        """Accept new client connections."""
        while self.is_running:
            try:
                client, addr = self.server_socket.accept()
                self.client_list.addItem(f"{addr}")
                threading.Thread(target=handle_client, args=(client,)).start()
            except:
                break

    def broadcast_message(self):
        """Send a broadcast message from the server."""
        message = self.message_input.text().strip()
        if message:
            broadcast(cipher.encrypt(f"SERVER: {message}".encode()))
            self.message_input.clear()
        else:
            QMessageBox.warning(self, "Warning", "Message cannot be empty!")


if __name__ == "__main__":
    init_db()
    app = QApplication([])
    server_gui = ServerGUI()
    server_gui.show()
    app.exec_()
